<a href="https://colab.research.google.com/github/markste-in/collatz/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import sympy
from sympy.ntheory import factorint
from multiprocessing import Pool
import psutil
import time
import os

In [3]:
print("CPU Count:", psutil.cpu_count())

CPU Count: 2


In [4]:
pool = Pool(psutil.cpu_count())

In [5]:
def collatz_serie(n : int) -> int:
    ret = [n]
    while (n>1):
        if n%2 == 0: 
            _n = int(n / 2)
        else: 
            _n = n*3+1
        n = _n
        ret.append(n)
    return ret

In [6]:
collatz_serie(19)

[19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]

In [7]:
xheader = ["Number", 
          "Length",
          "Max", 
          "IsEven",
          "nPrimes",
          "Smallest_Prime",
          "nSmallest_Prim",
          "Biggest_Prime",
          "nBiggest_Prime", 
          "isPrime"]

def all_fancy_calculations(serie):
        calculations = list()
        num_in_question = serie[0]
        fac_n = factorint(num_in_question)
        
        calculations.append(num_in_question)       # Starting number
        calculations.append(len(serie))     # Length of series
        calculations.append(max(serie))     # Max of number series
        calculations.append(num_in_question%2 ==0) # Is even?
        calculations.append(len(fac_n)) # how many primes
        calculations.append(min(fac_n)) # smallest prime
        calculations.append(fac_n[min(fac_n)]) # how often does the smallest prime appear
        calculations.append(max(fac_n)) # biggest prime
        calculations.append(fac_n[max(fac_n)]) # how often does the biggest prime appear
        calculations.append(sympy.isprime(num_in_question)) #is prime?
        
        return calculations
    

In [8]:
SMALLEST_NUMBER = 3 # since we calculate the previous serie the smallest number is actually one smaller (should not go below 2 since the return value of the collatz could be an empty list)

def create_dataset(start, stop):
    cz_series = pool.map(collatz_serie, range(start,stop+1))
    dataset = pool.map(all_fancy_calculations, cz_series)
    return pd.DataFrame(dataset)

In [9]:
# Save dataframe
def save_df(df):
  PATH = "gdrive/MyDrive/collatz_dbs/"
  os.makedirs(PATH,exist_ok=True)
  FILENAME = 'collatz_db_' + time.strftime("%Y%m%d-%H%M%S_") + str(np.random.randint(1e3,1e9)) + '.parquet'
  file = os.path.join(PATH,FILENAME)
  df.to_parquet(file)
  print("Saved to", file)

In [10]:
dist = int(1e5)
max_num = int(1e6)
start = 3
for end in range(start,max_num,dist):
  start_time = time.time()
  print('## Going from ',start, "to",start + dist, "##")
  df = create_dataset(start,start + dist)
  df.columns = xheader
  save_df(df)
  start = start + dist
  print("Iteration took", time.time()-start_time)

## Going from  3 to 100003 ##


Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.7/multiprocessing/

KeyboardInterrupt: ignored

In [ ]:
df.sample(5)

In [ ]:
1e10/1e6

In [ ]:
for i in range(3,int(1e6),int(1e5)):
  print(i)